# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

## Open Connection

In [1]:
import os
import re
import glob
import uuid
import psycopg2
import pandas as pd
import sql_queries as sq

In [3]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
conn = psycopg2.connect(
    user="admin",
    password="<password>",
    host="localhost", 
    port="5432",
    dbname="sparkifydb"
)
cur = conn.cursor()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

In [4]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def single_quote_converter(sentence):
    sentence = re.sub(r'\'', r'<single_quote_tag>', sentence)

    return sentence
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

## Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
song_files = get_files(os.path.join(os.getcwd(), "data/song_data"))

In [6]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(song_files)}")
print(f"Example index 0: {os.path.basename(song_files[0])}")

Total number of JSON files: 71
Example index 0: TRAAAAW128F429D538.json


In [7]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = song_files[0]

In [8]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
songs_df = pd.read_json(filepath, lines=True)
songs_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [9]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Artist ID, characters lenght: {len(songs_df['artist_id'].values[0])}")
print(f"Artist ID, characters lenght: {len(songs_df['song_id'].values[0])}")

Artist ID, characters lenght: 18
Artist ID, characters lenght: 18


### `artists` Table

#### Extract Data for Artists Table

- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = [
    "artist_id",
    "artist_name",
    "artist_location",
    "artist_latitude",
    "artist_longitude"
]
artist_data = songs_df[columns].copy()
artist_data["artist_name"] = artist_data["artist_name"].map(
    single_quote_converter, na_action='ignore'
)
artist_data["artist_location"] = artist_data["artist_location"].map(
    single_quote_converter, na_action='ignore'
)
artist_data = artist_data.drop_duplicates()
artist_data = artist_data.reset_index(drop=True)
print(f"Total number of artists:\n{artist_data.shape[0]}\n")
artist_data.head()
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

Total number of artists:
1



,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


##### Insert Record into Artist Table

Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
print("Print a query example (incomplete data, just a sample).\n")
query = sq.artist_table_insert(dataframe=artist_data[0:5], verbose=True)

Print a query example (incomplete data, just a sample).

SQL statement:
INSERT INTO artists
(artist_id, "name", location, latitude, longitude)
VALUES
    ('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', NULL, NULL)
ON CONFLICT (artist_id)
DO UPDATE SET
"name" = EXCLUDED."name",
location = EXCLUDED.location,
latitude = EXCLUDED.latitude,
longitude = EXCLUDED.longitude;




In [12]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = artist_data.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.artist_table_insert(
        dataframe=artist_data.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Send batch from idx '0'...


Run `test.ipynb` to see if you've successfully added a record to this table.

##### Results (DBeaver)

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

<!-- Original image size [563 164] -->
<br/><br/> <!-- Blank line -->
<div align="center">
  <img width="563" height="164" src="img/artists_results.png" />
</div>
<br/><br/> <!-- Blank line -->

### `songs` Table

#### Extract Data for Songs Table

- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [13]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["song_id", "title", "artist_id", "year", "duration"]
song_data = songs_df[columns].copy()
song_data["title"] = song_data["title"].map(
    single_quote_converter, na_action='ignore'
)
song_data = song_data.drop_duplicates()
song_data = song_data.reset_index(drop=True)
print(f"Total number of songs:\n{song_data.shape[0]}\n")
song_data.head()

Total number of songs:
1



,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn<single_quote_tag>t Mean To,ARD7TVE1187B99BFB1,0,218.93179


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.song_table_insert(dataframe=song_data.iloc[0:5], verbose=True)

SQL statement:
INSERT INTO songs
(song_id, title, artist_id, "year", duration)
VALUES
    ('SOMZWCG12A8C13C480', 'I Didn''t Mean To', 'ARD7TVE1187B99BFB1', 0, 218.93179)
ON CONFLICT (song_id)
DO UPDATE SET
title = EXCLUDED.title,
artist_id = EXCLUDED.artist_id,
"year" = EXCLUDED."year",
duration = EXCLUDED.duration;




In [15]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = song_data.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.song_table_insert(
        dataframe=song_data.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Send batch from idx '0'...


Run `test.ipynb` to see if you've successfully added a record to this table.

##### Results (DBeaver)

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

<!-- Original image size [569 162] -->
<br/><br/> <!-- Blank line -->
<div align="center">
  <img width="569" height="162" src="img/songs_results.png" />
</div>
<br/><br/> <!-- Blank line -->

## Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
log_files = get_files(os.path.join(os.getcwd(), "data/log_data"))

In [17]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
print(f"Total number of JSON files: {len(log_files)}")
print(f"Example index 0: {os.path.basename(log_files[0])}")

Total number of JSON files: 30
Example index 0: 2018-11-01-events.json


In [18]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filepath = log_files[0]

In [19]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
logs_df = pd.read_json(filepath, lines=True)
print(f"Total number of logs: {logs_df.shape[0]}")
logs_df.head()

Total number of logs: 15


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [20]:
logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         11 non-null     object 
 1   auth           15 non-null     object 
 2   firstName      15 non-null     object 
 3   gender         15 non-null     object 
 4   itemInSession  15 non-null     int64  
 5   lastName       15 non-null     object 
 6   length         11 non-null     float64
 7   level          15 non-null     object 
 8   location       15 non-null     object 
 9   method         15 non-null     object 
 10  page           15 non-null     object 
 11  registration   15 non-null     int64  
 12  sessionId      15 non-null     int64  
 13  song           11 non-null     object 
 14  status         15 non-null     int64  
 15  ts             15 non-null     int64  
 16  userAgent      15 non-null     object 
 17  userId         15 non-null     int64  
dtypes: float64(1

In [21]:
logs_df["page"].value_counts()

NextSong    11
Home         3
Upgrade      1
Name: page, dtype: int64

### Filter `log_data`

In [22]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df = logs_df[logs_df["page"] == 'NextSong'].copy()
del logs_df
print(f"Total number of logs with useful data:\n{filtered_logs_df.shape[0]}\n")
filtered_logs_df.head()

Total number of logs with useful data:
11



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


### `time` Table

#### Extract Data for Time Table

- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [23]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
filtered_logs_df['ts'] = pd.to_datetime(filtered_logs_df['ts'], unit='ms')
filtered_logs_df = filtered_logs_df.drop_duplicates()
print(f"Total number of logs with useful data:\n{filtered_logs_df.shape[0]}\n")
filtered_logs_df.head()

Total number of logs with useful data:
11



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,2018-11-01 21:11:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,2018-11-01 21:17:33.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [24]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_data = list(
    map(
        lambda item: [
            item.strftime('%Y-%m-%dT%H:%M:%S'),
            item.hour,
            item.day,
            item.week,
            item.month,
            item.year,
            item.weekday() + 1
        ],
        filtered_logs_df['ts'].copy()
    )
)
print(f"Print element list sample:\n{time_data[0]}\n")
column_labels = [
    'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'
]
print(f"Print column names:\n{column_labels}\n")
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#

Print element list sample:
['2018-11-01T21:01:46', 21, 1, 44, 11, 2018, 4]

Print column names:
['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']



In [25]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
time_list_dict = [dict(zip(column_labels, row)) for row in time_data]
print(f"Print first row dictionary sample:\n{time_list_dict[0]}\n")

Print first row dictionary sample:
{'start_time': '2018-11-01T21:01:46', 'hour': 21, 'day': 1, 'week': 44, 'month': 11, 'year': 2018, 'weekday': 4}



In [26]:
time_df = pd.DataFrame(time_list_dict)
time_df = time_df[column_labels]
time_df = time_df.drop_duplicates()
time_df = time_df.reset_index(drop=True)
print(f"Current dataframe dimensions:\n{time_df.shape}\n")
time_df.head()

Current dataframe dimensions:
(11, 7)



,start_time,hour,day,week,month,year,weekday
0,2018-11-01T21:01:46,21,1,44,11,2018,4
1,2018-11-01T21:05:52,21,1,44,11,2018,4
2,2018-11-01T21:08:16,21,1,44,11,2018,4
3,2018-11-01T21:11:13,21,1,44,11,2018,4
4,2018-11-01T21:17:33,21,1,44,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [28]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.time_table_insert(dataframe=time_df.iloc[0:5], verbose=True)

SQL statement:
INSERT INTO time
(start_time, hour, day, week, month, year, weekday)
VALUES
    ('2018-11-01T21:01:46', 21, 1, 44, 11, 2018, 4),
    ('2018-11-01T21:05:52', 21, 1, 44, 11, 2018, 4),
    ('2018-11-01T21:08:16', 21, 1, 44, 11, 2018, 4),
    ('2018-11-01T21:11:13', 21, 1, 44, 11, 2018, 4),
    ('2018-11-01T21:17:33', 21, 1, 44, 11, 2018, 4)
ON CONFLICT (start_time)
DO UPDATE SET
hour = EXCLUDED.hour,
day = EXCLUDED.day,
week = EXCLUDED.week,
month = EXCLUDED.month,
year = EXCLUDED.year,
weekday = EXCLUDED.weekday;




In [29]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = time_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.time_table_insert(
        dataframe=time_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Send batch from idx '0'...


Run `test.ipynb` to see if you've successfully added records to this table.

#### Results (DBeaver)

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

<!-- Original image size [570 316] -->
<br/><br/> <!-- Blank line -->
<div align="center">
  <img width="570" height="316" src="img/time_results.png" />
</div>
<br/><br/> <!-- Blank line -->

### `users` Table

#### Extract Data for Users Table

- Select columns for user ID, first name, last name, gender and level and set to `user_df`
- Drop duplicates by ID but keep last user found, this stores the current User level status
- Keep in mind that the fact table stores the current level status at hearing the song, but
  then the user can change its level.

In [30]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
columns = ["userId", "firstName", "lastName", "gender", "level"]
user_df = filtered_logs_df[columns].copy()
user_df["firstName"] = user_df["firstName"].map(
    single_quote_converter, na_action='ignore'
)
user_df["lastName"] = user_df["lastName"].map(
    single_quote_converter, na_action='ignore'
)
user_df["gender"] = user_df["gender"].str.upper()
user_df["userId"] = user_df["userId"].astype(str)
user_df = user_df.drop_duplicates()
user_df = user_df.drop_duplicates(subset='userId', keep='last')
user_df = user_df.reset_index(drop=True)
print(f"Total number of users:\n{user_df.shape}\n")
user_df.head()

Total number of users:
(4, 5)



,userId,firstName,lastName,gender,level
0,8,Kaylee,Summers,F,free
1,10,Sylvie,Cruz,F,free
2,26,Ryan,Smith,M,free
3,101,Jayden,Fox,M,free


#### Insert Records into Users Table

Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
query = sq.user_table_insert(dataframe=user_df.iloc[0:5], verbose=True)

SQL statement:
INSERT INTO users
(user_id, first_name, last_name, gender, level)
VALUES
    ('8', 'Kaylee', 'Summers', 'F', 'free'),
    ('10', 'Sylvie', 'Cruz', 'F', 'free'),
    ('26', 'Ryan', 'Smith', 'M', 'free'),
    ('101', 'Jayden', 'Fox', 'M', 'free')
ON CONFLICT (user_id)
DO UPDATE SET
first_name = EXCLUDED.first_name,
last_name = EXCLUDED.last_name,
gender = EXCLUDED.gender,
level = EXCLUDED.level;




In [32]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = user_df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.user_table_insert(
        dataframe=user_df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Send batch from idx '0'...


Run `test.ipynb` to see if you've successfully added records to this table.

#### Results (DBeaver)

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

<!-- Original image size [494 192] -->
<br/><br/> <!-- Blank line -->
<div align="center">
  <img width="494" height="192" src="img/users_results.png" />
</div>
<br/><br/> <!-- Blank line -->

### `songplays` Table

#### Extract Data and Songplays Table

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

> **length** in log files is the song duration in seconds

In [98]:
# insert songplay records
columns = [
    "ts",
    "userId",
    "level",
    "song",
    "artist",
    "length",
    "sessionId",
    "location",
    "userAgent"
]
df = filtered_logs_df[columns].copy()

# Format timestamp tp be upload in the database
df["ts"] = df["ts"].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Apply filtering of single quotes
df["song"] = df["song"].map(
    single_quote_converter, na_action='ignore'
)
df["artist"] = df["artist"].map(
    single_quote_converter, na_action='ignore'
)
df["location"] = df["location"].map(
    single_quote_converter, na_action='ignore'
)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df = df.reset_index(level=0)
print(f"Total number of songplay:\n{df.shape}\n")
df.head()

Total number of songplay:
(11, 10)



,index,ts,userId,level,song,artist,length,sessionId,location,userAgent
0,0,2018-11-01T21:01:46,8,free,You Gotta Be,Des<single_quote_tag>ree,246.30812,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,1,2018-11-01T21:05:52,8,free,Flat 55,Mr Oizo,144.03873,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
2,2,2018-11-01T21:08:16,8,free,Quem Quiser Encontrar O Amor,Tamba Trio,177.18812,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,3,2018-11-01T21:11:13,8,free,Eriatarka,The Mars Volta,380.42077,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,4,2018-11-01T21:17:33,8,free,Becoming Insane,Infected Mushroom,440.26730,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [99]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
# Test query creation
columns_in = ["index", "song", "artist", "length"]
query = sq.song_select(
    dataframe=df[columns_in].iloc[0:5],
    verbose=True
)

SQL statement:
SELECT
    song_artist.song_id song, --input_data.song
    song_artist.artist_id artist --input_data.artist
FROM (
    SELECT idx, song, artist, duration
    FROM (
        VALUES
        (0, 'You Gotta Be', 'Des''ree', 246.30812),
        (1, 'Flat 55', 'Mr Oizo', 144.03873),
        (2, 'Quem Quiser Encontrar O Amor', 'Tamba Trio', 177.18812),
        (3, 'Eriatarka', 'The Mars Volta', 380.42077),
        (4, 'Becoming Insane', 'Infected Mushroom', 440.2673)
    ) AS headers (idx, song, artist, duration)
) AS input_data
LEFT JOIN (
    SELECT
        songs.song_id,
        songs.title,
        songs.duration,
        artists.artist_id,
        artists."name"
    FROM songs
    LEFT JOIN artists
    ON songs.artist_id = artists.artist_id
) song_artist
ON input_data.song = song_artist.title
AND input_data.artist = song_artist."name"
AND input_data.duration = song_artist.duration
ORDER BY input_data.idx ASC;




In [101]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = df.shape[0]
batch_size = 5_000
columns_in = ["index", "song", "artist", "length"]
columns_out = ["song", "artist"]
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.song_select(
        dataframe=df[columns_in].iloc[index:index + batch_size]
    )
    temp_df = pd.read_sql(query, conn)
    df.loc[index:index + temp_df.shape[0], columns_out] = temp_df

df[columns_out] = df[columns_out].astype(str)
df[columns_out[0]] = df[columns_out[0]].map(
    lambda val: val if val != 'None' else 'nan'
)
df[columns_out[1]] = df[columns_out[1]].map(null_dict)
df = df.drop(columns=['index', 'length'])
df.head()

Send batch from idx '0'...


,ts,userId,level,song,artist,sessionId,location,userAgent
0,2018-11-01T21:01:46,8,free,nan,nan,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,2018-11-01T21:05:52,8,free,nan,nan,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
2,2018-11-01T21:08:16,8,free,nan,nan,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,2018-11-01T21:11:13,8,free,nan,nan,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,2018-11-01T21:17:33,8,free,nan,nan,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


#### Upload data

In [102]:
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
start_index = 0
end_index = df.shape[0]
batch_size = 5_000
for index in range(start_index, end_index, batch_size):
    print(f"Send batch from idx '{index}'...")
    query = sq.songplay_table_insert(
        dataframe=df.iloc[index:index + batch_size]
    )
    cur.execute(query)
    conn.commit()

Send batch from idx '0'...


Run `test.ipynb` to see if you've successfully added records to this table.

#### Results (DBeaver)

<!--
#|-----20--char-----||-----20--char-----||-----20--char-----||-doc-lim-||code|#
-->

<!-- Original image size [796 316] -->
<br/><br/> <!-- Blank line -->
<div align="center">
  <img width="796" height="316" src="img/songplays_results.png" />
</div>
<br/><br/> <!-- Blank line -->

## Close Connection to Sparkify Database

In [103]:
conn.close()

> Implement `etl.py`
> Use what you've completed in this notebook to implement `etl.py`.